In [ ]:
#alejandro briones
#chris kontsis
#sheil patel
#cse471 capstone 

#dev env completed
from google.colab import files
files.upload()

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d geomack/spotifyclassification

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('spotifyclassification.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
#import necesssary data analysis libraries

import numpy as np # linear algebra
import pandas as pd # data preprocessing
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn import datasets, linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

%matplotlib inline

data_frame = pd.read_csv("../content/data.csv")
data_frame = data_frame.drop("Unnamed: 0", axis="columns")
data_frame.head()

In [ ]:
chosen = ["energy", "liveness", "tempo", "valence", "loudness", "speechiness", "acousticness", "danceability", "instrumentalness"]
text1 = data_frame["artist"] + " - " + data_frame["song_title"]
text2 = text1.values

# X = data_frame.drop(droppable, axis=1).values
X = data_frame[chosen].values
y = data_frame["danceability"].values

min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

pca = PCA(n_components=3)
pca.fit(X)

X = pca.transform(X)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

trace = go.Scatter3d(
    x=X[:,0],
    y=X[:,1],
    z=X[:,2],
    text=text2,
    mode="markers",
    marker=dict(
        size=8,
        color=y
    )
)

fig = go.Figure(data=[trace])
py.iplot(fig, filename="test-graph")
fig.show()

In [ ]:
chosen = ["energy", "liveness", "tempo", "valence"]
text1 = data_frame["artist"] + " - " + data_frame["song_title"]
text2 = text1.values

# X = data_frame.drop(droppable, axis=1).values
X = data_frame[chosen].values
y = data_frame["loudness"].values

min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

pca = PCA(n_components=2)
pca.fit(X)

X = pca.transform(X)

fig = {
    "data": [
        {
            "x": X[:, 0],
            "y": X[:, 1],
            "text": text2,
            "mode": "markers",
            "marker": {"size": 8, "color": y}
        }
    ],
    "layout": {
        "xaxis": {"title": "How hard is this to dance to?"},
        "yaxis": {"title": "How metal is this?"}
    }
}

py.iplot(fig, filename="test-graph2")

In [ ]:
#Elbow test to determine number of clusters for k-means
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
selectedElbowFeatures = data_frame[['acousticness', 'danceability', 'duration_ms','energy','instrumentalness','key','liveness','loudness','speechiness','tempo','valence']]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(selectedElbowFeatures)

wcss = []

for i in range(1, 11):
    kmeans1 = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans1.fit(scaled_features)
    wcss.append(kmeans1.inertia_)

# Plotting the results onto a line graph
plt.figure(figsize=(10,5))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Beginning of fitting to model
# Kmeans clustering model
###Currently removing parameters to see if model performs better removed:'duration_ms'
selectedFeatures = data_frame[['acousticness', 'danceability', 'energy','instrumentalness','key','liveness','loudness','speechiness','tempo','valence']]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(selectedFeatures)

kmeans = KMeans(n_clusters = 7)
data_frame['cluster'] = kmeans.fit_predict(scaled_features)

def recommend_song(mood):

  mood_to_cluster = {
      'happy':0,
      'sad':1,
      'enegertic':2,
      'anxious':3,
      'depression':4,
      'calm':5,
      'romanctic':6
  }

  cluster = mood_to_cluster[mood]

  song_recommendation = data_frame[data_frame['cluster'] == cluster].sample(1)

  return song_recommendation

song = recommend_song('anxious')
data_frame

In [ ]:
K-Distance Graph for DBSCAN Cluster
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

min_samples = 2  # A starting point for min_samples

nearest_neighbors = NearestNeighbors(n_neighbors=min_samples)
neighbors = nearest_neighbors.fit(scaled_features)
distances, indices = neighbors.kneighbors(scaled_features)

# Sort the distances
sorted_distances = np.sort(distances[:, -1], axis=0)

# Plot the k-distance graph
plt.figure(figsize=(10, 5))
plt.plot(sorted_distances)
plt.xlabel('Points sorted by distance')
plt.ylabel('kth nearest distance')
plt.title('k-Distance Graph')
plt.grid(True)
plt.show()

In [ ]:
#DBSCAN Clustering
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.06, min_samples=2)
mood_score_sized = scaled_features_df['mood_score'].values.reshape(-1, 1)
data_frame['dbscan_cluster'] = dbscan.fit_predict(mood_score_sized)

# Check the number of clusters formed
n_clusters_ = len(set(data_frame['dbscan_cluster'])) - (1 if -1 in data_frame['dbscan_cluster'] else 0)
n_noise_ = list(data_frame['dbscan_cluster']).count(-1)

print(f'Estimated number of clusters: {n_clusters_}')
print(f'Estimated number of noise points: {n_noise_}')

In [ ]:
###silhoutte score

from sklearn.metrics import silhouette_score

# Assuming 'data_frame' is your original DataFrame and it now includes the 'cluster' column from KMeans
# Also assuming 'scaled_features_df' is your DataFrame of scaled features with the 'mood_score'

# Calculate silhouette score using the mood score and the cluster labels
silhouette_avg = silhouette_score(scaled_features_df[['mood_score']], data_frame['cluster'])

print(f'The average silhouette score is: {silhouette_avg}')


In [ ]:
#Determining qualaites of each cluster
cluster_to_mood = {
    0: "energetic",
    1: "happy",
    2: "romantic",
    3: "anxious",
    4: "depression",
    5: "sad",
    6: "calm",
}
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features)
kmeans = KMeans(n_clusters=7, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(principal_components)
plt.figure(figsize=(10,5))
scatter = plt.scatter(principal_components[:, 0], principal_components[:, 1], c=kmeans.labels_, cmap='rainbow', alpha=0.6)

# Add labels for each cluster
for i, txt in enumerate(kmeans.labels_):
    if txt in cluster_to_mood:
      plt.annotate(cluster_to_mood[txt], (principal_components[i, 0], principal_components[i, 1]), fontsize=9, alpha=1.0)
    else:
      print(f"Unexpected cluster label: {txt}")


# For better visualization, let's also add a legend
legend1 = plt.legend(*scatter.legend_elements(),
                    loc="upper right", title="Moods")
plt.gca().add_artist(legend1)

#plt.show()

for cluster_num in range(7):  # Change this if you have a different number of clusters
    cluster_songs = data_frame[data_frame['cluster'] == cluster_num]
    print(f"Cluster {cluster_num} ({cluster_to_mood.get(cluster_num, 'Unknown')}):")
    for index, row in cluster_songs.iterrows():
        print(f"{row['artist']} - {row['song_title']}")
    print("\n")

In [ ]:
#Performing linear regression on each cluster
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

clusters = data_frame['cluster'].unique()

cluster_models = {}

for cluster in clusters:
    cluster_data = data_frame[data_frame['cluster'] == cluster]

    # Splitting features and target variable
    # Assuming 'features' are your song features and 'target' is your target variable
    X = cluster_data[['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo' ]]  # Add all your features here
    y = cluster_data['valence'].valuess
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    y = y.reshape(-1, 1)
    y_scaled = scaler.fit_transform(y)
    # Splitting data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    # Training a linear regression model for this cluster
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Store the trained model
    cluster_models[cluster] = model

    # Predict & evaluate (optional)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Cluster {cluster} - MSE: {mse:.2f}")



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_regression_results(y_true, y_pred, model_name):
    """
    Plot regression results: True vs Predicted values and residuals.

    y_true: array-like, true target values
    y_pred: array-like, predicted target values by the model
    model_name: str, name of the regression model
    """
    residuals = y_true - y_pred

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

    # Plot true vs predicted values
    axes[0].scatter(y_true, y_pred, alpha=0.6)
    axes[0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=2)
    axes[0].set_xlabel('True Value')
    axes[0].set_ylabel('Predicted Value')
    axes[0].set_title(f'{model_name}: True vs Predicted')

    # Plot residuals
    axes[1].scatter(y_true, residuals, alpha=0.6)
    axes[1].hlines(0, y_true.min(), y_true.max(), 'k', lw=2)
    axes[1].set_xlabel('True Value')
    axes[1].set_ylabel('Residual')
    axes[1].set_title(f'{model_name}: Residuals')

    plt.tight_layout()
    plt.show()

##plot_regression_results(y, y_pred, cluster[0])